In [3]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
interview_data = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx",
    sheet_name="Interview_Data",
    skiprows=lambda x: x == 1,
)
# interview_data.head()

phq_score = pd.read_excel(
    "../data/DAIC_demographic_data.xlsx",
    sheet_name="Metadata_mapping"
)

# phq_score.head()

,Participant_ID,PHQ_Score
0,300,2
1,301,3
2,302,4
3,303,0
4,304,6


The following step was a workaround to prevent the end of file error due to unclosed inverted commas. What the code is doing - opening and reading each file, reading each line and checking for lines that start with " but does not end with " (i.e., checking for unclosed quotes).

In [ ]:
# preprocessing the csv files to remove the unclosed inverted commas


def clean_csv(file_path, output_path):
    with open(file_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            if line.startswith('"') and not line.endswith('"\n'):
                line = line[1:]
            outfile.write(line)


def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            clean_csv(input_path, output_path)

input_dir = "../data/E-DAIC_Transcripts"
output_dir = "../data/E-DAIC_Transcripts_cleaned"

process_directory(input_dir, output_dir)

In [31]:
cleaned_transcripts_path = "../data/E-DAIC_Transcripts_cleaned"

transcripts = {}

# loop through each file in the folder, load it, and store the content
for filename in os.listdir(cleaned_transcripts_path):
    if filename.endswith("_Transcript.csv"):
        participant_id = filename.split("_")[0] # extract the participant id
        file_path = os.path.join(cleaned_transcripts_path, filename)

        df = pd.read_csv(file_path)
        transcripts[participant_id] = df

# transcripts
transcripts["386"]

,Text
0,might have pulled something that
1,I'm going to bring the great thanks so much
2,and please
3,are you okay with this yes
4,oh I'm fine I'm a little tired but I found ou...
...,...
76,yeah well after college people usually many p...
77,thank you goodbye
78,oh that was that was fast
79,but I didn't never said there wasn't any like...


In [ ]:
# combine all the text data for each participant into a single string
combined_transcripts = {}

for participant_id, transcript in transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript["Text"].astype(str))

# removing extra spaces caused by newlines
for participant_id, transcript in combined_transcripts.items():
    combined_transcripts[participant_id] = " ".join(transcript.split())

# combined_transcripts["386"]

"might have pulled something that I'm going to bring the great thanks so much and please are you okay with this yes oh I'm fine I'm a little tired but I found out my thyroid is I think acting up so where are you from originally I was born in Canada but I've lived in California most of my life so and it's gray today too so the gray weather makes you kind of see no sluggish oh my gosh years and years ago not at all I don't think I can text to her for a long time well the air fares have gone up quite a bit in the last few years gas prices have gone up so travel as much as it's a fun thing to do it's cost-prohibitive so oh that's see everything has plus and minuses cold-weather not as much sun so I think I'd prefer getting more sun and having the warm-weather so but other than that it's it's very pretty back their natural scenery in but I like it here you know the traffic is kind of heavy but that's a small thing to deal with why did I move to LA because my family came here and started to 

## Extracting Language Features
### _CountVectorizer_

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_features = count_vectorizer.fit_transform(combined_transcripts.values())

print(f"Shape of the feature: {count_features.shape}")

count_feature_names = count_vectorizer.get_feature_names_out()
print(count_feature_names)

Shape of the feature: (190, 8702)
['00' '02' '04' ... 'zuma' 'zurich' 'zz']


The row in the `shape` attribute represents the number of documents - in our case, we have $190$ transcripts, which matches with the output. The column value is the number of unique words in the vocabulary. In our case, we have $8702$ unique words, which is shown by the array printed at the end.

In [41]:
# checking the frequency of the words

# converting the token count matrix to a dataframe for easy manipulation
count_df = pd.DataFrame(count_features.toarray(), columns=count_feature_names)

word_counts = count_df.sum(axis=0)

word_counts = word_counts.sort_values(ascending=False)
print(f"Word frequency:\n{word_counts}")

Word frequency:
and             8170
to              7149
the             6051
that            5181
it              4646
                ... 
incorporate        1
inconsistent       1
included           1
incidents          1
zz                 1
Length: 8702, dtype: int64


### _TF-IDF Vectorizer_